# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 42 new papers today
          17 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/16 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2209.07538


extracting tarball to tmp_2209.07538...

 done.
Retrieving document from  https://arxiv.org/e-print/2209.07539


extracting tarball to tmp_2209.07539...

...

✔ → 118957:\section{Supporting figures}
  ↳ 121017:end


Retrieving document from  https://arxiv.org/e-print/2209.07962


extracting tarball to tmp_2209.07962...

 done.


/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Found documentclass in tmp_2209.07962/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'bibliography' from 'tmp_2209.07962/bibliography.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'latexdefs' from 'tmp_2209.07962/latexdefs.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.07931-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.07931) | **The impact of dynamic pressure bumps on the observational properties of  protoplanetary disks**  |
|| <mark>Jochen Stadler</mark>, et al. -- incl., <mark>Matías Gárate</mark> |
|*Appeared on*| *2022-09-19*|
|*Comments*| *22 pages, 15 figures. To be published in Astronomy & Astrophysics*|
|**Abstract**| Over the last years, large (sub-)millimetre surveys of protoplanetary disks have well constrained the demographics of disks, such as their millimetre luminosities, spectral indices, and disk radii. Additionally, several high-resolution observations have revealed an abundance of substructures in the disks dust continuum. The most prominent are ring like structures, likely due to pressure bumps trapping dust particles. The origins and characteristics of these bumps, nevertheless, need to be further investigated. The purpose of this work is to study how dynamic pressure bumps affect observational properties of protoplanetary disks. We further aim to differentiate between the planetary- versus zonal flow-origin of pressure bumps. We perform one-dimensional gas and dust evolution simulations, setting up models with varying pressure bump features. We subsequently run radiative transfer calculations to obtain synthetic images and the different quantities of observations. We find that the outermost pressure bump determines the disks dust size across different millimetre wavelengths. Our modelled dust traps need to form early (< 0.1 Myr), fast (on viscous timescales), and must be long lived (> Myr) to obtain the observed high millimetre luminosities and low spectral indices of disks. While the planetary bump models can reproduce these observables irrespectively of the opacity prescription, the highest opacities are needed for the zonal flow bump model to be in line with observations. Our findings favour the planetary- over the zonal flow-origin of pressure bumps and support the idea that planet formation already occurs in early class 0-1 stages of circumstellar disks. The determination of the disks effective size through its outermost pressure bump also delivers a possible answer to why disks in recent low-resolution surveys appear to have the same sizes across different millimetre wavelengths. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.07962-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.07962) | **Resolved SPLASH Chemodynamics in Andromeda's PHAT Stellar Halo and Disk:  On the Nature of the Inner Halo Along the Major Axis**  |
|| Ivanna Escala, et al. -- incl., <mark>Lara Cullinane</mark>, <mark>Benjamin F. Williams</mark> |
|*Appeared on*| *2022-09-19*|
|*Comments*| *Submitted to AJ. Conclusions on page 20. 18 figures, 2 tables, 4 appendices*|
|**Abstract**| Stellar kinematics and metallicity are key to exploring formation scenarios for galactic disks and halos. In this work, we characterized the relationship between kinematics and photometric metallicity along the line-of-sight to M31's disk. We combined optical HST/ACS photometry from the Panchromatic Hubble Andromeda Treasury (PHAT) survey with Keck/DEIMOS spectra from the Spectroscopic and Photometric Landscape of Andromeda's Stellar Halo (SPLASH) survey. The resulting sample of 3536 individual red giant branch stars spans 4-19 projected kpc, making it a useful probe of both the disk and inner halo. We separated these stars into disk and halo populations by modeling the line-of-sight velocity distributions as a function of position across the disk region, where 70.9% stars have a high likelihood of belonging to the disk and 17.1% to the halo. Although stellar halos are typically thought to be metal-poor, the kinematically identified halo contains a significant population of stars (29.4%) with disk-like metallicity ([Fe/H]$_{\rm phot} \sim -0.10$). This metal-rich halo population lags the gaseous disk to a similar extent as the rest of the halo, indicating that it does not correspond to a canonical thick disk. Its properties are inconsistent with those of tidal debris originating from the Giant Stellar Stream merger event. Moreover, the halo is chemically distinct from the phase-mixed component previously identified along the minor axis (i.e., away from the disk), implying contributions from different formation channels. These metal-rich halo stars provide direct chemodynamical evidence in favor of the previously suggested "kicked-up'' disk population in M31's inner stellar halo. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.07538-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.07538) | **The rebrightening of AT2018fyk as a repeating partial tidal disruption  event**  |
|| T. Wevers, et al. -- incl., <mark>M. Guolo</mark>, <mark>Z. Liu</mark> |
|*Appeared on*| *2022-09-19*|
|*Comments*| *Submitted to ApJ Letters*|
|**Abstract**| Stars that interact with supermassive black holes (SMBHs) can either be completely or partially destroyed by tides. In a partial tidal disruption event (TDE) the high-density core of the star remains intact, and the low-density, outer envelope of the star is stripped and feeds a luminous accretion episode. The TDE AT2018fyk, with an inferred black hole mass of $10^{7.7\pm0.4}$ M$_{\odot}$, experienced an extreme dimming event at X-ray (factor of $>$6000) and UV (factor $\sim$15) wavelengths $\sim$500--600 days after discovery. Here we report on the re-emergence of these emission components roughly 1200 days after discovery. We find that the source properties are similar to those of the pre-dimming accretion state, suggesting that the accretion flow was rejuvenated to a similar state. We propose that a repeating partial TDE, where the partially disrupted star is on a $\sim 600$ day orbit about the SMBH and is periodically stripped of mass during each pericenter passage, powers its unique lightcurve. This scenario provides a plausible explanation for AT2018fyk's overall properties, including the rapid dimming event and the rebrightening at late times. We also provide testable predictions for the behavior of the accretion flow in the future: if the second encounter was also a partial disruption then we predict another strong dimming event around day 1800 (August 2023), and a subsequent rebrightening around day 2400 (March 2025). This source provides strong evidence of the partial disruption of a star by a SMBH. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.07885-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.07885) | **Probing the Galactic halo with RR Lyrae stars $-$ III. The chemical and  kinematic properties of the stellar halo**  |
|| <mark>Gaochao Liu</mark>, et al. -- incl., <mark>Huawei Zhang</mark> |
|*Appeared on*| *2022-09-19*|
|*Comments*| *15 pages, 14 figures, MNRAS accepted*|
|**Abstract**| Based on a large spectroscopic sample of $\sim$ 4,300 RR Lyrae stars with metallicity, systemic radial velocity and distance measurements, we present a detailed analysis of the chemical and kinematic properties of the Galactic halo. Using this sample, the metallicity distribution function (MDF) as a function of $r$ and the velocity anisotropy parameter $\beta$ profiles (for different metallicity populations) are derived for the stellar halo. Both the chemical and kinematic results suggest that the Galactic halo is composed of two distinct parts, the inner halo and outer halo. The cutoff radius ($\sim$ 30 kpc) is similar to the previous break radius found in the density distribution of the stellar halo. We find that the inner part is dominated by a metal-rich population with extremely radial anisotropy ($\beta \sim 0.9$). These features are in accordance with those of ``{\it Gaia}-Enceladus-Sausage'' (GES) and we attribute this inner halo component as being dominantly composed of stars deposited from this ancient merged satellite. We find that GES probably has a slightly negative metallicity gradient. The metal-poor populations in the inner halo are characterized as a long-tail in MDF with an anisotropy of $\beta \sim 0.5$, which is similar to that of the outer part. The MDF for the outer halo is very broad with several weak peaks and the value of $\beta$ is around 0.5 for all metallicities. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.07542-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.07542) | **Toward an understanding of the properties of neural network approaches  for supernovae light curve approximation**  |
|| <mark>Mariia Demianenko</mark>, et al. |
|*Appeared on*| *2022-09-19*|
|*Comments*| *Submitted to MNRAS. 14 pages, 6 figures, 9 tables*|
|**Abstract**| The modern time-domain photometric surveys collect a lot of observations of various astronomical objects, and the coming era of large-scale surveys will provide even more information. Most of the objects have never received a spectroscopic follow-up, which is especially crucial for transients e.g. supernovae. In such cases, observed light curves could present an affordable alternative. Time series are actively used for photometric classification and characterization, such as peak and luminosity decline estimation. However, the collected time series are multidimensional, irregularly sampled, contain outliers, and do not have well-defined systematic uncertainties. Machine learning methods help extract useful information from available data in the most efficient way. We consider several light curve approximation methods based on neural networks: Multilayer Perceptrons, Bayesian Neural Networks, and Normalizing Flows, to approximate observations of a single light curve. Tests using both the simulated PLAsTiCC and real Zwicky Transient Facility data samples demonstrate that even few observations are enough to fit networks and achieve better approximation quality than other state-of-the-art methods. We show that the methods described in this work have better computational complexity and work faster than Gaussian Processes. We analyze the performance of the approximation techniques aiming to fill the gaps in the observations of the light curves, and show that the use of appropriate technique increases the accuracy of peak finding and supernova classification. In addition, the study results are organized in a Fulu Python library available on GitHub, which can be easily used by the community. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error Could not find figure (a)</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.07700-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.07700) | **The Effects of Space Weather on Flight Delays**  |
|| Y. Wang, et al. -- incl., <mark>W. Y. Zhang</mark>, <mark>Z. Li</mark>, <mark>J. P. Guo</mark> |
|*Appeared on*| *2022-09-19*|
|*Comments*| *submitted to science advances*|
|**Abstract**| Although the sun is really far away from us, some solar activities could still influence the performance and reliability of space-borne and ground-based technological systems on Earth. Those time-varying conditions in space caused by the sun are also called space weather, as the atmospheric conditions that can affect weather on the ground. It is known that aviation activities can be affected during space weather events, but the exact effects of space weather on aviation are still unclear. Especially how the flight delays, the top topic concerned by most people, will be affected by space weather has never been thoroughly researched. By analyzing huge amount of flight data (~5X106 records), for the first time, we demonstrate that space weather events could have systematically modulating effects on flight delays. The average arrival delay time and 30-minute delay rate during space weather events are significantly increased by 81.34% and 21.45% respectively compared to those during quiet periods. The evident negative correlation between the yearly flight regularity rate and the yearly mean total sunspot number during 22 years also confirms such delay effects. Further studies indicate that the interference in communication and navigation caused by geomagnetic field fluctuations and ionospheric disturbances associated with the space weather events will increase the flight delay time and delay rate. These results expand the traditional field of space weather research and could also provide us with brand new views for improving the flight delay predications. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.07701-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.07701) | **Characteristics of Flight Delays during Solar Flares**  |
|| X. H. Xu, et al. -- incl., <mark>W. Y. Zhang</mark>, <mark>Z. Li</mark>, <mark>J. P. Guo</mark> |
|*Appeared on*| *2022-09-19*|
|*Comments*| *submitted to APJL*|
|**Abstract**| Solar flare is one of the severest solar activities on the sun, and it has many important impacts on the near-earth space. It has been found that flight arrival delays will increase during solar flare. However, the detailed intrinsic mechanism of how solar flares influence the delays is still unknown. Based on 5-years huge amount of flight data, here we comprehensively analyze the flight departure delays during 57 solar flares. It is found that the averaged flight departure delay time during solar flares increased by 20.68% (7.67 min) compared to those during quiet periods. It is also shown that solar flare related flight delays reveal apparent time and latitude dependencies. Flight delays during dayside solar flares are more serious than those during nightside flares, and the longer (shorter) delays tend to occur in the lower (higher) latitude airport. Further analyses suggest that flight delay time and delay rate would be directly modulated by the solar intensity (soft X-ray flux) and the Solar Zenith Angle. For the first time, these results indicate that the communication interferences caused by solar flares will directly affect flight departure delay time and delay rate. This work also expands our conventional understandings to the impacts of solar flares on human society, and it could also provide us with brand new views to help prevent or cope with flight delays. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.07703-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.07703) | **Analyses of Flight Time During Solar Proton Events and Solar Flares**  |
|| X. H. Xu, et al. -- incl., <mark>Z. Li</mark>, <mark>W. Y. Zhang</mark> |
|*Appeared on*| *2022-09-19*|
|*Comments*| *submitted to Scientific Reports*|
|**Abstract**| Analyzing the effects of space weather on aviation is a new and developing topic. It has been commonly accepted that the flight time of the polar flights may increase during solar proton events because the flights have to change their route to avoid the high-energy particles. However, apart from such phenomenon, researches related to the flight time during space weather events is very rare. Based on the analyses of 39 representative international air routes around westerlies, it is found that 97.44% (94.87%) of the commercial airplanes on the westbound (eastbound) air routes reveal shorter (longer) flight time during solar proton events compared to those during quiet periods, and the averaged magnitude of change in flight time is ~10 min or 0.21%-4.17% of the total flight durations. Comparative investigations reassure the certainty of such phenomenon that the directional differences in flight time are still incontrovertible regardless of over-land routes (China-Europe) or over-sea routes (China-Western America). Further analyses suggest that the solar proton events associated atmospheric heating will change the flight durations by weakening certain atmospheric circulations, such as the polar jet stream. While the polar jet stream will not be obviously altered during solar flares so that the directional differences in flight time are not found. Besides the conventional space weather effects already known, this paper is the first report that indicates a distinct new scenario of how the solar proton events affect flight time. These analyses are also important for aviation since our discoveries could help the airways optimize the air routes to save passenger time costs, reduce fuel costs and even contribute to the global warming issues. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2209.07931.md
    + _build/html/tmp_2209.07931/./figures/SizeLumRel_1mm_lowhighres_TazzariComp.png
    + _build/html/tmp_2209.07931/./figures/SizeLumRel_3mm_lowhighres_TazzariComp.png
    + _build/html/tmp_2209.07931/./figures/SpecIndex_R68_lowres_Tazzari.png
    + _build/html/tmp_2209.07931/./figures/MDust_Rgap.png
    + _build/html/tmp_2209.07931/./figures/MDust_Amplitude.png
    + _build/html/tmp_2209.07931/./figures/MDust_Tgrow.png
    + _build/html/tmp_2209.07931/./figures/OptDepth_Rgap.png
    + _build/html/tmp_2209.07931/./figures/OptDepth_Amplitude.png
    + _build/html/tmp_2209.07931/./figures/OptDepth_Tgrow.png
    + _build/html/tmp_2209.07931/./figures/Dynamic_Sigmas_Compared_1Myrs_new1.png
    + _build/html/tmp_2209.07931/./figures/Dynamic_Masses_Time_log.png
exported in  _build/html/2209.07962.md
    + _build/html/tmp_2209.07962/figures/splash_mdf_rgb_lowextinction_medianAv.png
    + _build/html/tmp_2209.07962/figures/splash_membership_coloredbypna

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand$</div>



<div id="title">

# The impact of dynamic pressure bumps on the observational properties of protoplanetary disks

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2209.07931-b31b1b.svg)](https://arxiv.org/abs/2209.07931)<mark>Appeared on: 2022-09-19</mark> - _22 pages, 15 figures. To be published in Astronomy & Astrophysics_

</div>
<div id="authors">

<mark><mark>Jochen Stadler</mark></mark>, et al. -- incl., <mark><mark>Matías Gárate</mark></mark>

</div>
<div id="abstract">

**Abstract:** Over the last years, large (sub-)millimetre surveys of protoplanetary disks in different star forming regions have well constrained the demographics of disks, such as their millimetre luminosities, spectral indices, and disk radii. Additionally, several high-resolution observations have revealed an abundance of substructures in the disk's dust continuum. The most prominent are ring like structures, which are likely caused by pressure bumps trapping dust particles. The origins and characteristics of these pressure bumps, nevertheless, need to be further investigated.The purpose of this work is to study how dynamic pressure bumps affect observational properties of protoplanetary disks. We further aim to differentiate between the planetary- versus zonal flow-origin of pressure bumps.We perform one-dimensional gas and dust evolution simulations, setting up models with varying pressure bump features, including their amplitude and location, growth time, and number of bumps. We subsequently run radiative transfer calculations to obtain synthetic images, from which we obtain the different quantities of observations.We find that the outermost pressure bump determines the disk's dust size across different millimetre wavelengths and confirm that the observed dust masses of disks with optically thick inner bumps ($<40$au) are underestimated by up to an order of magnitude. Our modelled dust traps need to form early (< 0.1 Myr), fast (on viscous timescales), and must be long lived (> Myr) to obtain the observed high millimetre luminosities and low spectral indices of disks. While the planetary bump models can reproduce these observables irrespectively of the opacity prescription, the highest opacities are needed for the dynamic bump model, which mimics zonal flows in disks, to be in line with observations.Our findings favour the planetary- over the zonal flow-origin of pressure bumps and support the idea that planet formation already occurs in early class 0-1 stages of circumstellar disks. The determination of the disk's effective size through its outermost pressure bump also delivers a possible answer to why disks in recent low-resolution surveys appear to have the same sizes across different millimetre wavelengths.

</div>

<div id="div_fig1">

<img src="tmp_2209.07931/./figures/SizeLumRel_1mm_lowhighres_TazzariComp.png" alt="Fig3.1" width="33%"/><img src="tmp_2209.07931/./figures/SizeLumRel_3mm_lowhighres_TazzariComp.png" alt="Fig3.2" width="33%"/><img src="tmp_2209.07931/./figures/SpecIndex_R68_lowres_Tazzari.png" alt="Fig3.3" width="33%"/>

**Figure 3. -** Size-luminosity relations and spectral indices dependent on disk sizesSize-luminosity relations and spectral indices dependent on disk sizes. In \textbf{a) + b)} the fluxes are divided by the cosine of the inclination (i = 20°).  The plotted 68\% emission disk radii are found to decrease from low (0.30") to high (0.04") resolution for all models irrespective of wavelength which is highlighted by lines connecting both radii. The grey dashed lines show the relations for Lupus at $\lambda=$1.3 mm and $\lambda=$3.0 mm found by \cite{Tazzari2021_sizes}, where the grey area represents the 68\% confidence interval around the median. \textbf{c)} Spectral index ($\alpha_\mathrm{1.3-3.0mm}$) over disk effective size (R$_\mathrm{d, 68\%}$) at $\lambda=$1.3 mm with 0.30" resolution, black dots show Lupus data also from \cite{Tazzari2021_sizes}. (*fig:Tazzari_relations*)

</div>
<div id="div_fig2">

<img src="tmp_2209.07931/./figures/MDust_Rgap.png" alt="Fig8.1" width="16%"/><img src="tmp_2209.07931/./figures/MDust_Amplitude.png" alt="Fig8.2" width="16%"/><img src="tmp_2209.07931/./figures/MDust_Tgrow.png" alt="Fig8.3" width="16%"/><img src="tmp_2209.07931/./figures/OptDepth_Rgap.png" alt="Fig8.4" width="16%"/><img src="tmp_2209.07931/./figures/OptDepth_Amplitude.png" alt="Fig8.5" width="16%"/><img src="tmp_2209.07931/./figures/OptDepth_Tgrow.png" alt="Fig8.6" width="16%"/>

**Figure 8. -** Dust mass and optical depth parameter plots. \textsl{(Upper)} Total dust mass (blue), dust mass for grains larger than 0.1 mm (orange) and observed dust mass (green) plotted over the gap location, bump amplitude and bump growth time (from a) - c)). \textsl{(Lower)} Absorption and observed peak optical depth also in dependence on bump model parameters. The \textit{Inner Bump} model reaches $\tau_\mathrm{abs}=35$ which is why we used a logarithmic scale in subplot a) (*fig:Mdust_tau_effect*)

</div>
<div id="div_fig3">

<img src="tmp_2209.07931/./figures/Dynamic_Sigmas_Compared_1Myrs_new1.png" alt="Fig2.1" width="50%"/><img src="tmp_2209.07931/./figures/Dynamic_Masses_Time_log.png" alt="Fig2.2" width="50%"/>

**Figure 2. -** Surface density and dust masses plots of dynamic bumps models. \textbf{a)} Gas (dashed line) and dust (solid line) surface densities at 1 Myr of evolution for the different dynamic bumps models and the model without any bumps for comparison. \textbf{b)} Total dust mass of each model over the disk lifetime. (*fig:dynamic_bumps*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\nsplash}{6405}$
$\newcommand{\fkickedupdiskdorman}{5.2\% \pm 1.2\%}$
$\newcommand{\fmrtotstat}{6.5\% \pm 1.4\%}$
$\newcommand{\fmrtot}{5.5\%}$
$\newcommand{\fkickedupdiskexcess}{6.8\% \pm 0.4\%}$
$\newcommand{\fdisktotstat}{75.0\% \pm 4.5\%}$
$\newcommand{\fehphotshiftfourtotwelve}{-0.26}$
$\newcommand{\medfehphotneshelf}{-0.16}$
$\newcommand{\medfehphotminhalo}{-0.15}$
$\newcommand{\fehphotshiftnoinnerfield}{-0.29}$
$\newcommand{\frachostnbody}{98.9\%}$
$\newcommand{\ndiskrgb}{2443}$
$\newcommand{\nhalorgb}{657}$
$\newcommand{\nmixedrgb}{436}$
$\newcommand{\fehphotdiskrgbmr}{-0.10}$
$\newcommand{\fehphotdiskrgbmp}{-0.89}$
$\newcommand{\fehphotdiskrgbmrsig}{0.23}$
$\newcommand{\fehphotdiskrgbmpsig}{0.53}$
$\newcommand{\fehphotdiskrgbmrfrac}{70\%}$
$\newcommand{\fehphotdiskrgbmpfrac}{30\%}$
$\newcommand{\fehphothalorgbmr}{-0.16}$
$\newcommand{\fehphothalorgbmp}{-0.88}$
$\newcommand{\fehphothalorgbmrsig}{0.22}$
$\newcommand{\fehphothalorgbmpsig}{0.45}$
$\newcommand{\fehphothalorgbmrfrac}{50\%}$
$\newcommand{\fehphothalorgbmpfrac}{50\%}$
$\newcommand{\percentsupersolarhalo}{14\%}$
$\newcommand{\fcontamhalo}{12.5\%}$
$\newcommand{\fcontamdisk}{11.0\%}$
$\newcommand{\fehphotmedhalorgbcontamcorr}{-0.49^{+0.003}_{-0.009}}$
$\newcommand{\fehphotmeddiskrgbcontamcorr}{-0.16^{+0.002}_{-0.004}}$
$\newcommand{\fehphotavghalorgbcontamcorr}{-0.57 \pm 0.003}$
$\newcommand{\fehphotavgdiskrgbcontamcorr}{-0.30 \pm 0.003}$
$\newcommand{\gradientdisk}{-0.0190}$
$\newcommand{\gradientdiskerr}{0.0002}$
$\newcommand{\gradienthalo}{-0.0051}$
$\newcommand{\gradienthaloerr}{0.0004}$
$\newcommand{\interceptdisk}{+0.060}$
$\newcommand{\interceptdiskerr}{0.005}$
$\newcommand{\intercepthalo}{-0.279}$
$\newcommand{\intercepthaloerr}{0.004}$
$\newcommand{\gradientdisklowext}{-0.0220}$
$\newcommand{\gradientdisklowexterr}{0.0004}$
$\newcommand{\gradienthalolowext}{-0.0075}$
$\newcommand{\gradienthalolowexterr}{0.0004}$
$\newcommand{\interceptdisklowext}{+0.119}$
$\newcommand{\interceptdisklowexterr}{0.005}$
$\newcommand{\intercepthalolowext}{-0.242}$
$\newcommand{\intercepthalolowexterr}{0.006}$
$\newcommand{\gradientdiskdust}{-0.0185}$
$\newcommand{\gradientdiskdusterr}{0.0002}$
$\newcommand{\gradienthalodust}{-0.0090}$
$\newcommand{\gradienthalodusterr}{0.0003}$
$\newcommand{\interceptdiskdust}{+0.084}$
$\newcommand{\interceptdiskdusterr}{0.003}$
$\newcommand{\intercepthalodust}{-0.200}$
$\newcommand{\intercepthalodusterr}{0.004}$
$\newcommand{\gradientdiskfid}{-0.019^{+0.001}_{-0.003}}$
$\newcommand{\fracmrhalofid}{55.1\%}$
$\newcommand{\fracmrhalocontamcorr}{49.2\%}$
$\newcommand{\fracmrhalocontambydisk}{5.9\%}$
$\newcommand{\fracpkickfid}{29.4\%}$
$\newcommand{\fracpkickcontamcorr}{25.6\%}$
$\newcommand{\fracpkickcontam}{3.8\%}$
$\newcommand{\muvhalomr}{-266.3}$
$\newcommand{\muverrhalomr}{5.6}$
$\newcommand{\muxhalomr}{-0.12}$
$\newcommand{\muxerrhalomr}{0.02}$
$\newcommand{\sigvhalomr}{46.0}$
$\newcommand{\sigverrhalomr}{4.7}$
$\newcommand{\sigxhalomr}{0.19}$
$\newcommand{\sigxerrhalomr}{0.02}$
$\newcommand{\rhalomr}{0.26}$
$\newcommand{\rerrhalomr}{0.10}$
$\newcommand{\fmrhalo}{0.26}$
$\newcommand{\fmrerrhalo}{0.03}$
$\newcommand{\muvhalomp}{-322.9}$
$\newcommand{\muverrhalomp}{6.7}$
$\newcommand{\sigvhalomp}{139.3}$
$\newcommand{\sigverrhalomp}{4.5}$
$\newcommand{\muxhalomp}{-0.66}$
$\newcommand{\muxerrhalomp}{0.03}$
$\newcommand{\sigxhalomp}{0.51}$
$\newcommand{\sigxerrhalomp}{0.02}$
$\newcommand{\rhalomp}{0.08}$
$\newcommand{\rerrhalomp}{0.04}$
$\newcommand{\admeddiskall}{61.7^{+0.8}_{-0.7}}$
$\newcommand{\admedmixedall}{61.1^{+1.9}_{-1.5}}$
$\newcommand{\admedhaloall}{66.2^{+0.8}_{-2.1}}$
$\newcommand{\admedhalomrall}{63.1^{+3.4}_{-2.6}}$
$\newcommand{\percentmrhalo}{72.3\%}$
$\newcommand{\percentmrdisk}{80.0\%}$
$\newcommand{\percentmrmixed}{84.2\%}$
$\newcommand{\admeddisk}{68.2^{+1.0}_{-0.6}}$
$\newcommand{\admedmixed}{74.8^{+0.8}_{-1.6}}$
$\newcommand{\admedhalo}{75.2^{+1.4}_{-1.2}}$
$\newcommand{\admedhalomr}{77.9^{+1.2}_{-1.1}}$
$\newcommand{\addiskhalodiffsig}{2.9}$
$\newcommand{\addiskmixeddiffsig}{2.5}$
$\newcommand{\adhalomrdiffsig}{1.1}$
$\newcommand{\adhalomrdiffsigrest}{1.7}$
$\newcommand{\addiskhalodiffsigall}{1.6}$
$\newcommand{\fehphotmeddiskrgb}{-0.19}$
$\newcommand{\fehphotmedmixedrgb}{-0.16}$
$\newcommand{\fehphotmedhalorgb}{-0.41}$
$\newcommand{\fehphotmeddiskagb}{-0.07}$
$\newcommand{\fehphotmedmixedagb}{+0.14}$
$\newcommand{\fehphotmedhaloagb}{+0.01}$
$\newcommand{\verrmed}{7.1}$
$\newcommand{\nblended}{7}$
$\newcommand{\ngoodrv}{247}$
$\newcommand{\meddeltaewna}{2.5}$
$\newcommand{\nmembersplash}{4930}$
$\newcommand{\meanewand}{0.54}$
$\newcommand{\stdewand}{1.00}$
$\newcommand{\meanewmw}{2.27}$
$\newcommand{\stdewmw}{1.96}$
$\newcommand{\percentnonasplash}{51.6\%}$
$\newcommand{\muhaloone}{-258.2}$
$\newcommand{\muhalotwo}{-268.7}$
$\newcommand{\muhalothree}{-238.8}$
$\newcommand{\sighaloone}{134.4}$
$\newcommand{\sighalotwo}{135.3}$
$\newcommand{\sighalothree}{117.5}$
$\newcommand{\percentdisksplash}{70.9}$
$\newcommand{\percenthalosplash}{17.1}$
$\newcommand{\percentmixedsplash}{12.0}$
$\newcommand{\medradneone}{6.3}$
$\newcommand{\medradnetwo}{10.3}$
$\newcommand{\medradnethree}{14.4}$
$\newcommand{\fehphotmedallrgb}{-0.18}$
$\newcommand{\fehphotmedlowextrgb}{-0.21}$
$\newcommand{\fehphotdiffagblowext}{-0.01}$
$\newcommand{\fehphotdiffrgbcorr}{-0.03}$
$\newcommand{\fehphotdiffagbcorr}{-0.15}$
$\newcommand{\sigmaavmed}{0.30}$
$\newcommand{\diskscaleheight}{0.77}$
$\newcommand{\ngiantslowext}{2706}$
$\newcommand{\nrgblowext}{2080}$
$\newcommand{\fracoldagb}{67.5\%}$
$\newcommand{\nagbdustcorr}{556}$
$\newcommand{\nrgbdustcorr}{3536}$
$\newcommand{\fehphoterrmed}{0.03}$
$\newcommand{\medrederr}{0.004}$
$\newcommand{\medblueerr}{0.015}$
$\newcommand{\difffehphotrgb}{-0.22}$
$\newcommand{\difffehphotagb}{+0.48}$
$\newcommand{\nrgbsplashmwcorr}{3910}$
$\newcommand{\nrgbsplashmwcorrrheb}{3389}$
$\newcommand{\ewna}{EW_{\rm Na}}$
$\newcommand{\fehphot}{[Fe/H]_{\rm phot}}$
$\newcommand{\loggphot}{\log g_{\rm phot}}$
$\newcommand{\teffphot}{T_{\rm eff,phot}}$
$\newcommand{\logg}{\log g}$
$\newcommand{\teff}{T_{\rm eff}}$
$\newcommand{\feh}{[Fe/H]}$
$\newcommand{\alphafe}{[\alpha/Fe]}$
$\newcommand{\vhelio}{v_{\rm helio}}$
$\newcommand{\rproj}{R_{\rm proj}}$
$\newcommand{\fehphotinit}{[Fe/H]_{\rm phot, init}}$
$\newcommand{\fehphoterr}{\delta[Fe/H]_{\rm phot}}$
$\newcommand{\ewsum}{\Sigma{\rm Ca}}$
$\newcommand{\fehcat}{[Fe/H]_{\rm spec}}$
$\newcommand{\avdal}{A_{V, {\rm Dal}}}$
$\newcommand{\sigav}{\sigma_V}$
$\newcommand{\fred}{f_{\rm red}}$
$\newcommand{\pdisk}{p_{\rm disk}}$
$\newcommand{\Msun}{M_{\odot}}$
$\newcommand{\kms}{km s^{-1}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\nsplash$}{6405}$
$\newcommand{$\fkickedupdiskdorman$}{5.2\% \pm 1.2\%}$
$\newcommand{$\fmrtotstat$}{6.5\% \pm 1.4\%}$
$\newcommand{$\fmrtot$}{5.5\%}$
$\newcommand{$\fkickedupdiskexcess$}{6.8\% \pm 0.4\%}$
$\newcommand{$\fdisktotstat$}{75.0\% \pm 4.5\%}$
$\newcommand{$\fehphotshiftfourtotwelve$}{-0.26}$
$\newcommand{$\medfehphotneshelf$}{-0.16}$
$\newcommand{$\medfehphotminhalo$}{-0.15}$
$\newcommand{$\fehphotshiftnoinnerfield$}{-0.29}$
$\newcommand{$\frachostnbody$}{98.9\%}$
$\newcommand{$\ndiskrgb$}{2443}$
$\newcommand{$\nhalorgb$}{657}$
$\newcommand{$\nmixedrgb$}{436}$
$\newcommand{$\fehphotdiskrgbmr$}{-0.10}$
$\newcommand{$\fehphotdiskrgbmp$}{-0.89}$
$\newcommand{$\fehphotdiskrgbmr$sig}{0.23}$
$\newcommand{$\fehphotdiskrgbmp$sig}{0.53}$
$\newcommand{$\fehphotdiskrgbmr$frac}{70\%}$
$\newcommand{$\fehphotdiskrgbmp$frac}{30\%}$
$\newcommand{$\fehphothalorgbmr$}{-0.16}$
$\newcommand{$\fehphothalorgbmp$}{-0.88}$
$\newcommand{$\fehphothalorgbmr$sig}{0.22}$
$\newcommand{$\fehphothalorgbmp$sig}{0.45}$
$\newcommand{$\fehphothalorgbmr$frac}{50\%}$
$\newcommand{$\fehphothalorgbmp$frac}{50\%}$
$\newcommand{$\percentsupersolarhalo$}{14\%}$
$\newcommand{$\fcontamhalo$}{12.5\%}$
$\newcommand{$\fcontamdisk$}{11.0\%}$
$\newcommand{$\fehphotmedhalorgbcontamcorr$}{-0.49^{+0.003}_{-0.009}}$
$\newcommand{$\fehphotmeddiskrgbcontamcorr$}{-0.16^{+0.002}_{-0.004}}$
$\newcommand{$\fehphotavghalorgbcontamcorr$}{-0.57 \pm 0.003}$
$\newcommand{$\fehphotavgdiskrgbcontamcorr$}{-0.30 \pm 0.003}$
$\newcommand{$\gradientdisk$}{-0.0190}$
$\newcommand{$\gradientdisk$err}{0.0002}$
$\newcommand{$\gradienthalo$}{-0.0051}$
$\newcommand{$\gradienthalo$err}{0.0004}$
$\newcommand{$\interceptdisk$}{+0.060}$
$\newcommand{$\interceptdisk$err}{0.005}$
$\newcommand{$\intercepthalo$}{-0.279}$
$\newcommand{$\intercepthalo$err}{0.004}$
$\newcommand{$\gradientdisk$lowext}{-0.0220}$
$\newcommand{$\gradientdisk$lowexterr}{0.0004}$
$\newcommand{$\gradienthalo$lowext}{-0.0075}$
$\newcommand{$\gradienthalo$lowexterr}{0.0004}$
$\newcommand{$\interceptdisk$lowext}{+0.119}$
$\newcommand{$\interceptdisk$lowexterr}{0.005}$
$\newcommand{$\intercepthalo$lowext}{-0.242}$
$\newcommand{$\intercepthalo$lowexterr}{0.006}$
$\newcommand{$\gradientdisk$dust}{-0.0185}$
$\newcommand{$\gradientdisk$dusterr}{0.0002}$
$\newcommand{$\gradienthalo$dust}{-0.0090}$
$\newcommand{$\gradienthalo$dusterr}{0.0003}$
$\newcommand{$\interceptdisk$dust}{+0.084}$
$\newcommand{$\interceptdisk$dusterr}{0.003}$
$\newcommand{$\intercepthalo$dust}{-0.200}$
$\newcommand{$\intercepthalo$dusterr}{0.004}$
$\newcommand{$\gradientdisk$fid}{-0.019^{+0.001}_{-0.003}}$
$\newcommand{$\fracmrhalofid$}{55.1\%}$
$\newcommand{$\fracmrhalocontamcorr$}{49.2\%}$
$\newcommand{$\fracmrhalocontambydisk$}{5.9\%}$
$\newcommand{$\fracpkickfid$}{29.4\%}$
$\newcommand{$\fracpkickcontamcorr$}{25.6\%}$
$\newcommand{$\fracpkickcontam$}{3.8\%}$
$\newcommand{$\muvhalomr$}{-266.3}$
$\newcommand{$\muverrhalomr$}{5.6}$
$\newcommand{$\muxhalomr$}{-0.12}$
$\newcommand{$\muxerrhalomr$}{0.02}$
$\newcommand{$\sigvhalomr$}{46.0}$
$\newcommand{$\sigverrhalomr$}{4.7}$
$\newcommand{$\sigxhalomr$}{0.19}$
$\newcommand{$\sigxerrhalomr$}{0.02}$
$\newcommand{$\rhalomr$}{0.26}$
$\newcommand{$\rerrhalomr$}{0.10}$
$\newcommand{$\fmrhalo$}{0.26}$
$\newcommand{$\fmrerrhalo$}{0.03}$
$\newcommand{$\muvhalomp$}{-322.9}$
$\newcommand{$\muverrhalomp$}{6.7}$
$\newcommand{$\sigvhalomp$}{139.3}$
$\newcommand{$\sigverrhalomp$}{4.5}$
$\newcommand{$\muxhalomp$}{-0.66}$
$\newcommand{$\muxerrhalomp$}{0.03}$
$\newcommand{$\sigxhalomp$}{0.51}$
$\newcommand{$\sigxerrhalomp$}{0.02}$
$\newcommand{$\rhalomp$}{0.08}$
$\newcommand{$\rerrhalomp$}{0.04}$
$\newcommand{$\admeddiskall$}{61.7^{+0.8}_{-0.7}}$
$\newcommand{$\admedmixedall$}{61.1^{+1.9}_{-1.5}}$
$\newcommand{$\admedhaloall$}{66.2^{+0.8}_{-2.1}}$
$\newcommand{$\admedhalomrall$}{63.1^{+3.4}_{-2.6}}$
$\newcommand{$\percentmrhalo$}{72.3\%}$
$\newcommand{$\percentmrdisk$}{80.0\%}$
$\newcommand{$\percentmrmixed$}{84.2\%}$
$\newcommand{$\admeddisk$}{68.2^{+1.0}_{-0.6}}$
$\newcommand{$\admedmixed$}{74.8^{+0.8}_{-1.6}}$
$\newcommand{$\admedhalo$}{75.2^{+1.4}_{-1.2}}$
$\newcommand{$\admedhalo$mr}{77.9^{+1.2}_{-1.1}}$
$\newcommand{$\addiskhalodiffsig$}{2.9}$
$\newcommand{$\addiskmixeddiffsig$}{2.5}$
$\newcommand{$\adhalomrdiffsig$}{1.1}$
$\newcommand{$\adhalomrdiffsig$rest}{1.7}$
$\newcommand{$\addiskhalodiffsig$all}{1.6}$
$\newcommand{$\fehphotmeddiskrgb$}{-0.19}$
$\newcommand{$\fehphotmedmixedrgb$}{-0.16}$
$\newcommand{$\fehphotmedhalorgb$}{-0.41}$
$\newcommand{$\fehphotmeddiskagb$}{-0.07}$
$\newcommand{$\fehphotmedmixedagb$}{+0.14}$
$\newcommand{$\fehphotmedhaloagb$}{+0.01}$
$\newcommand{$\verrmed$}{7.1}$
$\newcommand{$\nblended$}{7}$
$\newcommand{$\ngoodrv$}{247}$
$\newcommand{$\meddeltaewna$}{2.5}$
$\newcommand{$\nmembersplash$}{4930}$
$\newcommand{$\meanewand$}{0.54}$
$\newcommand{$\stdewand$}{1.00}$
$\newcommand{$\meanewmw$}{2.27}$
$\newcommand{$\stdewmw$}{1.96}$
$\newcommand{$\percentnonasplash$}{51.6\%}$
$\newcommand{$\muhaloone$}{-258.2}$
$\newcommand{$\muhalotwo$}{-268.7}$
$\newcommand{$\muhalothree$}{-238.8}$
$\newcommand{$\sighaloone$}{134.4}$
$\newcommand{$\sighalotwo$}{135.3}$
$\newcommand{$\sighalothree$}{117.5}$
$\newcommand{$\percentdisksplash$}{70.9}$
$\newcommand{$\percenthalosplash$}{17.1}$
$\newcommand{$\percentmixedsplash$}{12.0}$
$\newcommand{$\medradneone$}{6.3}$
$\newcommand{$\medradnetwo$}{10.3}$
$\newcommand{$\medradnethree$}{14.4}$
$\newcommand{$\fehphotmedallrgb$}{-0.18}$
$\newcommand{$\fehphotmedlowextrgb$}{-0.21}$
$\newcommand{$\fehphotdiffagblowext$}{-0.01}$
$\newcommand{$\fehphotdiffrgbcorr$}{-0.03}$
$\newcommand{$\fehphotdiffagbcorr$}{-0.15}$
$\newcommand{$\sigmaavmed$}{0.30}$
$\newcommand{$\diskscaleheight$}{0.77}$
$\newcommand{$\ngiantslowext$}{2706}$
$\newcommand{$\nrgblowext$}{2080}$
$\newcommand{$\fracoldagb$}{67.5\%}$
$\newcommand{$\nagbdustcorr$}{556}$
$\newcommand{$\nrgbdustcorr$}{3536}$
$\newcommand{$\fehphoterrmed$}{0.03}$
$\newcommand{$\medrederr$}{0.004}$
$\newcommand{$\medblueerr$}{0.015}$
$\newcommand{$\difffehphotrgb$}{-0.22}$
$\newcommand{$\difffehphotagb$}{+0.48}$
$\newcommand{$\nrgbsplashmwcorr$}{3910}$
$\newcommand{$\nrgbsplashmwcorr$rheb}{3389}$
$\newcommand{$\ewna$}{EW_{\rm Na}}$
$\newcommand{$\fehphot$}{[Fe/H]_{\rm phot}}$
$\newcommand{$\loggphot$}{\log g_{\rm phot}}$
$\newcommand{$\teffphot$}{T_{\rm eff,phot}}$
$\newcommand{$\logg$}{\log g}$
$\newcommand{$\teff$}{T_{\rm eff}}$
$\newcommand{$\feh$}{[Fe/H]}$
$\newcommand{$\alphafe$}{[\alpha/Fe]}$
$\newcommand{$\vhelio$}{v_{\rm helio}}$
$\newcommand{$\rproj$}{R_{\rm proj}}$
$\newcommand{$\fehphot$init}{[Fe/H]_{\rm phot, init}}$
$\newcommand{$\fehphot$err}{\delta[Fe/H]_{\rm phot}}$
$\newcommand{$\ewsum$}{\Sigma{\rm Ca}}$
$\newcommand{$\feh$cat}{[Fe/H]_{\rm spec}}$
$\newcommand{$\avdal$}{A_{V, {\rm Dal}}}$
$\newcommand{$\sigav$}{\sigma_V}$
$\newcommand{$\fred$}{f_{\rm red}}$
$\newcommand{$\pdisk$}{p_{\rm disk}}$
$\newcommand{$\Msun$}{M_{\odot}}$
$\newcommand{$\kms$}{km s^{-1}}$</div>



<div id="title">

# Resolved SPLASH Chemodynamics in Andromeda's PHAT Stellar Halo and Disk: On the Nature of the Inner Halo Along the Major Axis

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2209.07962-b31b1b.svg)](https://arxiv.org/abs/2209.07962)<mark>Appeared on: 2022-09-19</mark> - _Submitted to AJ. Conclusions on page 20. 18 figures, 2 tables, 4 appendices_

</div>
<div id="authors">

Ivanna Escala, et al. -- incl., <mark><mark>Lara Cullinane</mark></mark>, <mark><mark>Benjamin F. Williams</mark></mark>

</div>
<div id="abstract">

**Abstract:** Stellar kinematics and metallicity are key to exploring formation scenarios for galactic disks and halos. In this work, we characterized the relationship between kinematics and photometric metallicityalong the line-of-sight to M31's disk.We combined optical HST/ACS photometry from the Panchromatic Hubble Andromeda Treasury (PHAT) survey with Keck/DEIMOSspectrafrom the Spectroscopic and Photometric Landscape of Andromeda's Stellar Halo (SPLASH) survey. The resulting sample of$\nrgbdustcorr$individual red giant branch stars spans 4--19 projected kpc, making it a useful probe of both the disk and inner halo.We separated these stars into disk and halo populations by modeling the line-of-sight velocity distributions as a function of position across the disk region, where$\percentdisksplash$\%stars have a high likelihood of belonging to the disk and$\percenthalosplash$\%to the halo. Although stellar halos are typically thought to be metal-poor, the kinematically identified halo contains a significant population of stars ($\fracpkickfid$)with disk-like metallicity ($\fehphot$$\sim$\fehphotdiskrgbmr).This metal-rich halo population lags the gaseous disk to a similar extent as the rest of the halo, indicating that it does not correspond to a canonical thick disk. Itsproperties are inconsistent with those of tidal debris originating from the Giant Stellar Stream merger event.Moreover, the halo is chemically distinct from the phase-mixed component previously identified along the minor axis (i.e., away from the disk), implying contributions from different formation channels. These metal-rich halo stars provide direct chemodynamicalevidence in favor of the previously suggested "kicked-up" disk population in M31's inner stellar halo.

</div>

<div id="div_fig1">

<img src="tmp_2209.07962/figures/splash_mdf_rgb_lowextinction_medianAv.png" alt="Fig4" width="100%"/>

**Figure 4. -** $\fehphot$init distributions for all RGB stars (gray filled histogram) and RGB stars located in low extinction regions of the disk (black open histogram). We measured $\fehphot$init using 4 Gyr PARSEC RGB isochrones on the foreground-reddening corrected optical CMD (Section \ref{sec:phot_params}).
    We define low extinction regions using $f_{\rm red} \times A_{V, {\rm Dal}} < 0.25$(\citealt{Gregersen2015}). The median $\fehphot$init for all (low extinction) RGB stars is shown as a gray (black) arrow, where [Fe/H]$_{\rm phot, init, med}$ = $\fehphotmedallrgb$($\fehphotmedlowextrgb$). This implies that most RGB stars in SPLASH are located in front of the dust layer and not significantly reddened.
     (*fig:mdf_lowext*)

</div>
<div id="div_fig2">

<img src="tmp_2209.07962/figures/splash_membership_coloredbypna_v2.png" alt="Fig8" width="100%"/>

**Figure 8. -** 
    Membership determination for SPLASH targets with successful radial velocity measurements \citep{Dorman2012,Dorman2015} along the line-of-sight to M31's disk (Section \ref{sec:members}). (Left) Foreground-extinction-corrected (F475W, F814W) PHAT v2 CMD (\citealt{Dalcanton2012,Williams2014}; Section \ref{sec:phot}, \ref{sec:zinit}) for stars with (colored points) and without (grey points) Na I $\lambda$8190 doublet equivalent width ($\ewna$) measurements. We show 12 Gyr PARSEC RGB isochrones \citep{Marigo2017} with $\fehphot$ = $-2.0, -1.5, -1.0, -0.5$, 0 for reference, assuming $m-M = 24.45 \pm 0.05$\citep{Dalcanton2012}.
    Stars with $p_{\rm Na} \leq 0.25$, which are $\geq$3 times more likely to belong to the MW foreground than M31, are classified as non-members. We also exclude stars with colors bluer than the most metal-poor RGB isochrone. (Right) $\ewna$ versus heliocentric velocity ($\vhelio$).
    We show the median uncertainty in $\ewna$($\delta$\ewna = $\meddeltaewna$\AA) as a gray errorbar.
    Stars with $p_{\rm Na} \leq 0.25$ preferentially have blue colors or lie above the TRGB and have more positive $\vhelio$, all of which are properties characteristic of MW foreground dwarfs.
     (*fig:members*)

</div>
<div id="div_fig3">

<img src="tmp_2209.07962/figures/ne3_vmodel.png" alt="Fig9" width="100%"/>

**Figure 9. -** Heliocentric velocity distributions for region R3 (Section \ref{sec:disk_regions}). From left to right, the panels show the velocity distribution in each subregion (grey histograms; Figure \ref{fig:disk_regions}; Table \ref{tab:vmodel}), where R3$_1$ straddles the major axis and R3$_4$ is the most distant from the major axis. We also show the fitted velocity model for each subregion (thick purple lines; Section \ref{sec:disk_model}), which is composed of a stellar halo component \citep{Dorman2012} with fixed mean and dispersion but variable fractional contribution (dotted blue lines) and a disk component (dashed green lines). Figures \ref{fig:ne1} and \ref{fig:ne2} show the velocity distributions and models for R1 and R2. In general, the trends between the disk component velocity and $\Delta$P.A.\@ follow that expected for an inclined rotating disk, approaching M31's systemic velocity ($-300$\kms) with increasing $\Delta$P.A. \citep{Dorman2012}. The stellar halo component also becomes more dominant with increasing $\Delta$P.A.\@ (*fig:ne3*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

35  publications in the last 7 days.
	 _build/html/2209.07962.md
	 _build/html/2209.07931.md
	 _build/html/2209.06833.md
	 _build/html/2209.05814.md
	 _build/html/2209.05593.md
	 _build/html/2209.04304.md
	 _build/html/2209.03367.md
	 _build/html/2209.02872.md
	 _build/html/2209.02725.md
	 _build/html/2209.02722.md
	 _build/html/2209.02092.md
	 _build/html/2209.01125.md
	 _build/html/2209.00828.md
	 _build/html/2209.00363.md
	 _build/html/2208.14927.md
	 _build/html/2208.09335.md
	 _build/html/2208.08872.md
	 _build/html/2204.06393.md
	 _build/html/2204.03335.md
	 _build/html/2204.03253.md
	 _build/html/2204.02998.md
	 _build/html/2204.02109.md
	 _build/html/2204.02017.md
	 _build/html/2204.01824.md
	 _build/html/2204.01758.md
	 _build/html/2204.01245.md
	 _build/html/2204.00793.md
	 _build/html/2204.00342.md
	 _build/html/2203.16959.md
	 _build/html/2203.16856.md
	 _build/html/2203.16735.md
	 _build/html/2203.16734.md
	 _build/html/2203.16504.md
	 _build/html/2203.15822.md
	 _build/ht

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers